<h1>
<center>Hybrid Network [A3TGCN + LSTM]</center>
</h1>

In [ ]:

import numpy as np
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import tensorflow as tf
import pandas as pd
import pickle
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN2, DCRNN, TGCN2, TGCN,  GConvGRU, GConvLSTM, GCLSTM, A3TGCN
from torch.nn import Linear
from torch.nn import ReLU
import torch.nn as nn
from torch.nn.init import kaiming_uniform_
from tqdm import tqdm
from calflops import calculate_flops
from fvcore.nn import FlopCountAnalysis, parameter_count_table

Project_Path='Project-Path'

### Input data

In [ ]:
def Get_Paths(Project_Path):
    TestDataset=Project_Path+ '/Data/Test_Dataset_Graph.pkl'
    TestTargets=Project_Path+ '/Data/Test_Rate_Timeseries.csv'
    TestMask=Project_Path+ '/Data/Test_Mask.csv'
    TrainDataset =Project_Path+ '/Data/Dataset_Graph.pkl'
    TrainTargets=Project_Path+ '/Data/STS_Rate_Timeseries.csv'
    return TestDataset,TestTargets,TestMask,TrainDataset,TrainTargets

### Edges & edge weights

In [ ]:
def edge_weights(Project_Path):
    path = Project_Path+'/Data/EdgesDuration250.csv'
    weights = 'Duration'
    Edge_List=pd.read_csv(path,sep=',', index_col=0)
    Distance=np.array(Edge_List[weights])
    print(Distance.shape)
    Edge_List=Edge_List.drop([weights], axis=1)
    Edges=np.array(Edge_List)
    Edges=Edges.T
    print(Edges.shape)
    return Edges,Distance

In [ ]:
TestDataset,TestTargets,TestMask,TrainDataset,TrainTargets=Get_Paths(Project_Path)
Edges,Distance=edge_weights(Project_Path)

## Static Graph Temporal Signal

In [ ]:
with open(TrainDataset, 'rb') as inp:
    Train_Dataset = pickle.load(inp)
Name=['Date_Sin','Holidays','Capacity','temp','humidity','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Predictions']
for i in range (0,len(Train_Dataset)):
    Train_Dataset[i]=Train_Dataset[i].sort_values("Slot_id")
    Train_Dataset[i]=Train_Dataset[i].reset_index()
    Train_Dataset[i]=Train_Dataset[i].drop(['index'], axis=1)
    Train_Dataset[i]=Train_Dataset[i][Name]


Rate_Timeseries=np.array(Train_Dataset)
Rate_Timeseries=np.reshape(Rate_Timeseries, (len(Train_Dataset), 222,28,1))
print(Rate_Timeseries.shape)

Target=pd.read_csv(TrainTargets,sep=',', index_col=0)
Target=np.array(Target)
Target=Target.T
Target=np.reshape(Target, (len(Train_Dataset), 222,1))
print(Target.shape)

graTrain=StaticGraphTemporalSignal(edge_index=Edges,edge_weight=Distance,features=Rate_Timeseries,targets=Target)
print("Dataset type:  ", graTrain)
print(next(iter(graTrain)))

In [ ]:
with open(TestDataset, 'rb') as inp:
    Test_Dataset = pickle.load(inp)
Name=['Date_Sin','Holidays','Capacity','temp','humidity','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Predictions']
for i in range (0,len(Test_Dataset)):
    Test_Dataset[i]=Test_Dataset[i].sort_values("Slot_id")
    Test_Dataset[i]=Test_Dataset[i].reset_index()
    Test_Dataset[i]=Test_Dataset[i].drop(['index'], axis=1)
    Test_Dataset[i]=Test_Dataset[i][Name]

Rate_Timeseries=np.array(Test_Dataset)
Rate_Timeseries=np.reshape(Rate_Timeseries, (len(Test_Dataset), 222,28,1))
print(Rate_Timeseries.shape)

Target=pd.read_csv(TestTargets,sep=',', index_col=0)

Target=np.array(Target)
Target=Target.T
Target=np.reshape(Target, (len(Test_Dataset), 222,1))
print(Target.shape)

graTest=StaticGraphTemporalSignal(edge_index=Edges,edge_weight=Distance,features=Rate_Timeseries,targets=Target)
print("Dataset type:  ", graTest)
print(next(iter(graTest)))

### Test / Train split

In [ ]:
train_dataset=graTrain
test_dataset=graTest

print("Number of train buckets: ", len(set(train_dataset)))
print("Number of test buckets: ", len(set(test_dataset)))

### Dataloaders


In [ ]:
batch_size=64

In [ ]:
train_input = np.array(train_dataset.features) # (5074, 222, 24, 1)
train_target = np.array(train_dataset.targets) # (5074, 207, 1)
train_x_tensor = torch.from_numpy(train_input).type(torch.FloatTensor)  # (B, N, F, T)
train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor)  # (B, N, T)
train_dataset_new = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size=batch_size,drop_last=True)

In [ ]:
test_input = np.array(test_dataset.features) # (, 207, 2, 12)
test_target = np.array(test_dataset.targets) # (, 207, 12)
test_x_tensor = torch.from_numpy(test_input).type(torch.FloatTensor)# (B, N, F, T)
test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor) # (B, N, T)
test_dataset_new = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset_new, batch_size=batch_size,drop_last=True)

Loading the graph once because it's a static graph

In [ ]:
for snapshot in train_dataset:
    static_edge_index = snapshot.edge_index
    break;

## Hybrid Network  [ A3TGCN Model + LSTM]


In [ ]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size):
        super(TemporalGNN, self).__init__()
        
        self.tgnn = A3TGCN2(in_channels=node_features,  out_channels=128, periods=periods,batch_size=batch_size) # node_features=2, periods=12       
        self.hidden2 = Linear(128, 64)
        
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        
        self.lstm = nn.LSTM(64, 32, 1, batch_first=True, bidirectional=True)
        
        self.hidden4 = Linear(64, 16)
        kaiming_uniform_(self.hidden4.weight, nonlinearity='relu')
        self.act4 = ReLU()

        self.linear = torch.nn.Linear(16, periods)
        

    def forward(self, x, edge_index):
        h = self.tgnn(x, edge_index)
        h = F.relu(h)

        h0 = torch.zeros(2, h.size(0), 32).requires_grad_()
        c0 = torch.zeros(2, h.size(0), 32).requires_grad_()
          
        h = self.hidden2(h)
        h = self.act2(h)
        
        out, (hn, cn) = self.lstm(h, (h0.detach(), c0.detach()))
        
        out = self.hidden4(out)
        out = self.act4(out)
        
        out = self.linear(out) 
        return out

TemporalGNN(node_features=28, periods=1, batch_size=batch_size)

## Training and Evaluation Process

Cost & Error Function Calculation for raw Testset

In [ ]:
def GetTest_MAE_MSE(model,test_loader,metric_fn,TestMask):
    model.eval()
    step = 0
    total_loss = []
    Predictions=[]
    true=[]
    Test_Mask=pd.read_csv(TestMask,index_col=0)

    Test_Mask['New']=0
    Test_Mask=Test_Mask.T
    Test_Mask=Test_Mask.values.tolist()
    P=[]
    R=[]
    for encoder_inputs, labels in test_loader:
        # Get model predictions
        y_hat = model(encoder_inputs, static_edge_index)
        # Mean squared error
        for ii in range (0,len(y_hat)):
            Predictions.append(y_hat[ii])

        for kk in range (0,len(labels)):
            true.append(labels[kk])

        loss = metric_fn(y_hat, labels)
        total_loss.append(loss.item())
    
    for i in range(0,(len(Predictions))):
        for k in range (0,len(Test_Mask[0])):
            if Test_Mask[i][k]==1:
                P1=Predictions[i][k]
                P.append(float(P1))
                R1=true[i][k]
                R.append(float(R1))
    MAE=tf.keras.metrics.mean_absolute_error(R, P)
    MSE=tf.keras.metrics.mean_squared_error(R, P)
    return MAE,MSE

### Training for 25 epochs

In [ ]:
model = TemporalGNN(node_features=28, periods=1, batch_size=batch_size)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()
metric_fn=torch.nn.L1Loss()

model.train()
Test_MAE=[]
Test_MSE=[]
Train_Loss=[]
for epoch in range(1,25):
    step = 0
    loss_list = []
    for encoder_inputs, labels in tqdm(train_loader):
        y_hat = model(encoder_inputs, static_edge_index)         # Get model predictions
        loss = loss_fn(y_hat, labels) # Mean squared error #loss = torch.mean((y_hat-labels)**2)  sqrt to change it to rmse
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        step= step+ 1
        loss_list.append(loss.item())
    
    Loss=sum(loss_list)/len(loss_list)
    print("Epoch {} train MSE: {:.4f}".format(epoch, Loss))
    MAE,MSE=GetTest_MAE_MSE(model,test_loader,metric_fn,TestMask)
    Test_MAE.append(float(MAE))
    Test_MSE.append(float(MSE))
    Train_Loss.append(Loss)
    print("Test MAE: {:.4f}".format(MAE))
    print("Test MSE: {:.4f}".format(MSE))

### Experiments with Other Architectures 

In [ ]:
class TemporalGNN_RNN(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size, gcn, filter_size):
        super(TemporalGNN_RNN, self).__init__()
        
        if gcn == 'A3TGCN2':
            self.tgnn = A3TGCN2(in_channels=node_features, periods=periods, out_channels=64, batch_size=batch_size, cached=True)
        elif gcn == 'DCRNN':
            self.tgnn = DCRNN(in_channels=node_features, out_channels=64, K=periods)
        elif gcn == 'TGCN2':
            self.tgnn = TGCN2(in_channels=node_features, out_channels=64, batch_size=batch_size, cached=True)
        elif gcn == 'TGCN':
            self.tgnn = TGCN(in_channels=node_features, out_channels=64, cached=True)
        elif gcn == 'GConvGRU':
            self.tgnn = GConvGRU(in_channels=node_features, out_channels=64, K=filter_size)

        self.hidden2 = Linear(64, 32)        
        self.act2 = ReLU()
        
        self.lstm = torch.nn.LSTM(32, 16, 1, batch_first=True, bidirectional=True)
        
        self.hidden3 = Linear(32, 16)
        self.act3 = ReLU()        

        self.linear = torch.nn.Linear(16, periods)       

    def forward(self, x, edge_index, edge_weight):
        h = self.tgnn(x, edge_index, edge_weight)
        h = F.relu(h)

        h = self.hidden2(h)
        h = self.act2(h)
        
        h = self.hidden3(h)
        h = self.act3(h)
        
        out = self.linear(h) 
        return out

In [ ]:
class TemporalGNN_LSTM(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size, gcn, filter_size):
        super(TemporalGNN_LSTM, self).__init__()
        
        if gcn == 'GConvLSTM':
            self.tgnn = GConvLSTM(in_channels=node_features, out_channels=64, K=filter_size)
        elif gcn == 'GCLSTM':
            self.tgnn = GCLSTM(in_channels=node_features, out_channels=64, K=filter_size)

        self.hidden2 = Linear(64, 32)        
        self.act2 = ReLU()
        
        self.lstm = torch.nn.LSTM(32, 16, 1, batch_first=True, bidirectional=True)
            
        self.hidden3 = Linear(32, 16)
        self.act3 = ReLU()        

        self.linear = torch.nn.Linear(16, periods)       

    def forward(self, x, edge_index, edge_weight):
        h, _ = self.tgnn(x, edge_index, edge_weight)
        h = F.relu(h)

        h0 = torch.zeros(2, h.size(0), 16).requires_grad_()
        c0 = torch.zeros(2, h.size(0), 16).requires_grad_()
        
        h = self.hidden2(h)
        h = self.act2(h)
        
        h = self.hidden3(h)
        h = self.act3(h)
        
        out = self.linear(h) 
        return out

In [ ]:
gcn_list = ['A3TGCN2', 'DCRNN', 'TGCN', 'TGCN2', 'GConvGRU', 'GConvLSTM', 'GCLSTM']
recurent_type_list = ['RNN','TGCN2_BiDirectionalLSTM', 'TGCN2_LSTM', 'LSTM']
gcn = gcn_list[4]
recurent_type = recurent_type_list[0]
squeeze_input = True
filter_size = 3

In [ ]:
if (recurent_type == 'RNN') or (recurent_type == 'LSTM'):
    print('Graph Convolutional Network: ' + gcn)
else:
     print('Graph Convolutional Network: ' + recurent_type)
    
model, metrics_test = model_training(node_features, recurent_steps, lr, epochs, train_loader, test_loader, loss_fn,
                                     gcn, filter_size, squeeze_input, recurent_type, num_nodes)

In [ ]:
def model_training(node_features,batch_size,lr,epochs,train_loader,val_loader,loss_fn,
                   gcn, filter_size, squeeze_input,arch,num_nodes):
    if arch == 'RNN':
        model = TemporalGNN_RNN(node_features=node_features, periods=1, batch_size=batch_size,
                            gcn=gcn, filter_size=filter_size)
    
    elif arch == 'LSTM':
        model = TemporalGNN_LSTM(node_features=node_features, periods=1, batch_size=batch_size,


    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
    loss_fn = torch.nn.MSELoss()
    metric_fn=torch.nn.L1Loss()

    model.train()
    Test_MAE=[]
    Test_MSE=[]
    Train_Loss=[]
    for epoch in range(1,25):
        step = 0
        loss_list = []
        for encoder_inputs, labels in tqdm(train_loader):
            y_hat = model(encoder_inputs, static_edge_index)         # Get model predictions
            loss = loss_fn(y_hat, labels) # Mean squared error #loss = torch.mean((y_hat-labels)**2)  sqrt to change it to rmse
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            step= step+ 1
            loss_list.append(loss.item())

        Loss=sum(loss_list)/len(loss_list)
        print("Epoch {} train MSE: {:.4f}".format(epoch, Loss))
        MAE,MSE=GetTest_MAE_MSE(model,test_loader,metric_fn,TestMask)
        Test_MAE.append(float(MAE))
        Test_MSE.append(float(MSE))
        Train_Loss.append(Loss)
        print("Test MAE: {:.4f}".format(MAE))
        print("Test MSE: {:.4f}".format(MSE))

### Calculate Model Metrics (FLOPs,MACs,Params)

In [ ]:
model = TemporalGNN(node_features=28, periods=1, batch_size=batch_size)
batch_size = 64
input_shape = (batch_size, 5074, 222, 28)
flops, macs, params = calculate_flops(model=model, 
                                      input_shape=input_shape,
                                      output_as_string=True,
                                      output_precision=4)
print(" FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))

### Calculate Model Metrics (FLOPs,Params, Size) for State of the Art

In [ ]:
# Step 3: Define the TemporalGNN_RNN model
class TemporalGNN_RNN(nn.Module):
    def __init__(self, node_features, periods, batch_size, gcn, filter_size):
        super(TemporalGNN_RNN, self).__init__()

        if gcn == 'A3TGCN2':
            self.tgnn = A3TGCN2(in_channels=node_features, periods=periods, out_channels=64, batch_size=batch_size, cached=True)
        elif gcn == 'A3TGCN':
            self.tgnn = A3TGCN(in_channels=node_features, periods=64, out_channels=64, cached=True)
        elif gcn == 'DCRNN':
            self.tgnn = DCRNN(in_channels=node_features, out_channels=64, K=periods)
        elif gcn == 'TGCN2':
            self.tgnn = TGCN2(in_channels=node_features, out_channels=32, batch_size=batch_size, cached=True)
        elif gcn == 'TGCN':
            self.tgnn = TGCN(in_channels=node_features, out_channels=64, cached=True)
        elif gcn == 'GConvGRU':
            self.tgnn = GConvGRU(in_channels=node_features, out_channels=64, K=filter_size)

        self.hidden2 = Linear(64, 16)
        self.act2 = ReLU()

        self.linear = torch.nn.Linear(16, periods)

    def forward(self, x, edge_index, edge_weights):
        h = self.tgnn(x, edge_index, edge_weights)
        h = F.relu(h)

        h = self.hidden2(h)
        h = self.act2(h)

        out = self.linear(h)
        return out

# Step 4: Create a function to perform FLOPs and parameter analysis
def analyze_model(node_features, periods, batch_size, gcn, filter_size):
    model = TemporalGNN_RNN(node_features, periods, batch_size, gcn, filter_size)

    # Create sample input
    num_nodes = 222  # Example number of nodes in the graph
    edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2))  # Example edge indices
    edge_weights = torch.rand(num_nodes * 2)  # Example edge weights
    
    if gcn in ['A3TGCN2', 'A3TGCN', 'TGCN2', 'TGCN']:
        x = torch.rand(num_nodes, node_features)  # Example node features
    else:
        x = torch.rand(num_nodes, periods, node_features)  # Example node features for other models

    # Perform FLOPs and parameter analysis
    flop_counter = FlopCountAnalysis(model, (x, edge_index, edge_weights))
    flops = flop_counter.total()
    params = sum(p.numel() for p in model.parameters())
    param_table = parameter_count_table(model)

    # Calculate the model size in MB
    model_size_mb = params * 4 / (1024 ** 2)  # assuming float32 (4 bytes per parameter)

    # Print the results
    print(f"Model: {gcn}")
    print(f"Total FLOPs: {flops}")
    print(f"Total Parameters: {params}")
    print(param_table)
    print(f"Model Size: {model_size_mb:.2f} MB")
    print('-' * 40)

# Step 5: Analyze all model configurations
node_features = 28  # Example input feature dimension
periods = 10  # Example number of periods (output dimension)
batch_size = 32  # Example batch size
filter_size = 2  # Example filter size for GConvGRU

models = ['A3TGCN2', 'A3TGCN', 'DCRNN', 'TGCN2', 'TGCN', 'GConvGRU']
for model_name in models:
    analyze_model(node_features, periods, batch_size, model_name, filter_size)
